In [3]:
!pip show tensorflow
import tensorflow as tf

Name: tensorflow
Version: 2.3.4
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages
Requires: absl-py, astunparse, gast, google-pasta, grpcio, h5py, keras-preprocessing, numpy, opt-einsum, protobuf, six, tensorboard, tensorflow-estimator, termcolor, wheel, wrapt
Required-by: ei-for-tf, tensorflow-serving-api


# GPU Utilization Check

In [ ]:
!nvidia-smi --query-gpu=utilization.gpu --format=csv --loop=1

# GPU Check

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.test.gpu_device_name())

Num GPUs Available:  1
/device:GPU:0


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2664018565414981113,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 6679607833699018615
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7813347566678670722
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11141855360
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11374575465122862357
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"]

In [14]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
stepping	: 1
microcode	: 0xb00003e
cpu MHz		: 2337.262
cache size	: 46080 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid aperfmperf pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch cpuid_fault invpcid_single pti fsgsbase bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx xsaveopt
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa itlb_multihit
bogomips	: 4600.16
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtua

In [15]:
!cat /proc/meminfo

MemTotal:       62874696 kB
MemFree:        57379012 kB
MemAvailable:   58913060 kB
Buffers:          167588 kB
Cached:          1885400 kB
SwapCached:            0 kB
Active:          3650292 kB
Inactive:        1426160 kB
Active(anon):    3008664 kB
Inactive(anon):     3140 kB
Active(file):     641628 kB
Inactive(file):  1423020 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:                80 kB
Writeback:             0 kB
AnonPages:       3023620 kB
Mapped:           584820 kB
Shmem:              3148 kB
Slab:             123448 kB
SReclaimable:      92644 kB
SUnreclaim:        30804 kB
KernelStack:        4640 kB
PageTables:        12916 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    31437348 kB
Committed_AS:    5213256 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:    880640 kB
ShmemHugePages:  

# GPU vs CPU

In [13]:
%%time
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

with tf.device('gpu:0'):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    model.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=5)
    model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2206 - accuracy: 0.9336
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0963 - accuracy: 0.9702
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0694 - accuracy: 0.9772
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0542 - accuracy: 0.9825
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0583 - accuracy: 0.9828
CPU times: user 27.8 s, sys: 4.21 s, total: 32 s
Wall time: 21.7 s


In [12]:
%%time
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

with tf.device('/cpu:0'):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    model.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=5)
    model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2190 - accuracy: 0.9357
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0960 - accuracy: 0.9707
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0678 - accuracy: 0.9783
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0529 - accuracy: 0.9823
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.0640 - accuracy: 0.9811
CPU times: user 54.2 s, sys: 2.97 s, total: 57.1 s
Wall time: 23.3 s
